In [1]:
import numpy as np
import cv2
import matplotlib.pyplot as plt
from scipy.spatial.transform import Rotation
import utils

In [2]:
K1 = np.array([[1000, 0, 300],
             [0, 1000, 200],
             [0, 0, 1]])

R1 = np.identity(3)
t1 = np.array([[0.0, 0.0, 0.0]])

K2 = K1.copy()
R2 = Rotation.from_euler('xyz', [0.7, -0.5, 0.8]).as_matrix()
t2 = np.array([[0.2, 2.0, 1.0]])

In [3]:
Q = np.array([[1.0, 0.5, 4.0]]).T

p1 = utils.projectpoints(K1, R1, t1, Q)
p2 = utils.projectpoints(K2, R2, t2, Q)
print(p1)
print(p2)

[[550.]
 [325.]]
[[582.47256835]
 [185.98985776]]


In [4]:
p1_ = np.random.rand(1, 3)
p2_ = np.random.rand(1, 3)

print(np.cross(p1_, p2_))
print(p1_@utils.crossOp(p2_))
print(np.cross(p1_, p2_) == p1_@utils.crossOp(p2_))

[[ 0.15827765 -0.35170533 -0.01455361]]
[[ 0.15827765 -0.35170533 -0.01455361]]
[[ True  True  True]]


3.3

In [5]:
E = utils.crossOp(t2)@R2

F = np.linalg.inv(K2).T @ E @ np.linalg.inv(K1)
print(F)

[[ 3.29311881e-07  8.19396327e-07  1.79162592e-03]
 [ 5.15532551e-07 -8.76915984e-07  9.31426656e-05]
 [-1.29882755e-03  1.51951700e-03 -1.10072682e+00]]


3.4

In [6]:
l = F @ utils.PiInv(p1)
print(l / 1.3212389 * 5.285) #scale to get the solution shown in the exercise pdf

[[ 8.95628028e-03]
 [ 3.66754577e-04]
 [-5.28500021e+00]]


3.5

In [7]:
q1 = np.linalg.inv(K1)@utils.PiInv(p1)
q2 = np.linalg.inv(K2)@utils.PiInv(p2)

print(utils.PiInv(p2).T @ F @ utils.PiInv(p1))

[[4.4408921e-16]]


3.8

In [8]:
d = np.load('TwoImageDataCar.npy', allow_pickle=True).item()
im1 = d['im1']
R1 = d['R1']
t1 = d['t1']
im2 = d['im2']
R2 = d['R2']
t2 = d['t2']
K = d['K']
R = R2 @ R1.T
t = np.expand_dims(t2 - R2@R1.T@t1, axis=0)
E = utils.crossOp(t)@R
F = np.linalg.inv(K).T @ E @ np.linalg.inv(K)
print(F)

[[-1.50228281e-08 -3.45997421e-07 -3.47606501e-05]
 [-2.06767970e-07  3.96284278e-08 -9.29558240e-04]
 [ 2.61581163e-05  1.12168578e-03  1.17449076e-02]]


In [9]:
def DrawLine(l, shape, ax):
    #Checks where the line intersects the four sides of the image
    # and finds the two intersections that are within the frame
    def in_frame(l_im):
        q = np.cross(l.flatten(), l_im)
        q = q[:2]/q[2]
        if all(q>=0) and all(q+1<=shape[1::-1]):
            return q
    lines = [[1, 0, 0], [0, 1, 0], [1, 0, 1-shape[1]], [0, 1, 1-shape[0]]]
    P = [in_frame(l_im) for l_im in lines if in_frame(l_im) is not None]
    if (len(P)==0):
        print("Line is completely outside image")
    ax.plot(*np.array(P).T)

In [10]:
%matplotlib qt

fig, axes = plt.subplots(1, 2)

axes[0].imshow(im1)
axes[0].axis('off')

axes[1].imshow(im2)
axes[1].axis('off')

# From img1 to img2
#p = np.asarray(plt.ginput(1))
#l = F @ utils.PiInv(p.T)
#DrawLine(l, im2.shape, axes[1])

# From img2 to img1
p = np.asarray(plt.ginput(1))
l = (utils.PiInv(p.T).T @ F)
DrawLine(l, im2.shape, axes[0])

plt.show()

3.11

In [11]:
%matplotlib qt

fig, axes = plt.subplots(1, 2)

axes[0].imshow(im1)
axes[0].axis('off')

axes[1].imshow(im2)
axes[1].axis('off')

q1 = np.asarray(plt.ginput(1)).T
q2 = np.asarray(plt.ginput(1)).T

print("Original points:")
print(q1)
print(q2)
print()

q = np.concatenate((q1, q2), axis=1)
P = [utils.getP(K, R1, np.expand_dims(t1, axis=0)), utils.getP(K, R2, np.expand_dims(t2, axis=0))]
Q = utils.triangulate(q, P)

print("Reconstructed 3D point:")
print(utils.Pi(Q))
print()

q1_ = utils.Pi(P[0] @ Q)
q2_ = utils.Pi(P[1] @ Q)

print("After reprojection:")
print(q1_)
print(q2_)

Original points:
[[280.1375    ]
 [ 57.30615625]]
[[387.225     ]
 [ 58.19990625]]

Reconstructed 3D point:
[[-0.43217281]
 [-0.54361062]
 [ 1.74392106]]

After reprojection:
[[280.11115436]
 [ 55.6696984 ]]
[[387.33099885]
 [ 59.84308248]]


Quizzes

In [12]:
K = np.array([[900, 0, 1070], [0, 900, 610.0], [0, 0, 1]], float)
R1 = cv2.Rodrigues(np.array([-1.6, 0.3, -2.1]))[0]
t1 = np.array([0.0, 1.0, 3.0], float)
R2 = cv2.Rodrigues(np.array([-0.4, -1.3, -1.6]))[0]
t2 = np.array([0.0, 1.0, 6.0], float)
R3 = cv2.Rodrigues(np.array([2.5, 1.7, -0.4]))[0]
t3 = np.array([2.0, -7.0, 25.0], float)

p1 = np.array([[1046.0], [453.0]])
p2 = np.array([[1126.0], [671.0]])
p3 = np.array([[1165.0], [453.0]])

R = R2 @ R1.T
t = np.expand_dims(t2 - R2@R1.T@t1, axis=0)
E = utils.crossOp(t)@R
F = np.linalg.inv(K).T @ E @ np.linalg.inv(K)

dist = utils.PiInv(p2).T @ F @ utils.PiInv(p1)
print(dist)

[[-0.0931562]]


In [14]:
q = np.hstack((p1, p2, p3))

print("Original points:")
print(q)
print()

P = [utils.getP(K, R1, np.expand_dims(t1, axis=0)),
     utils.getP(K, R2, np.expand_dims(t2, axis=0)),
     utils.getP(K, R3, np.expand_dims(t3, axis=0))]
Q = utils.triangulate(q, P)

print("Reconstructed 3D point:")
print(utils.Pi(Q))
print()

q1_ = utils.Pi(P[0] @ Q)
q2_ = utils.Pi(P[1] @ Q)
q3_ = utils.Pi(P[2] @ Q)

print("After reprojection:")
print(q1_)
print(q2_)
print(q3_)

Original points:
[[1046. 1126. 1165.]
 [ 453.  671.  453.]]

Reconstructed 3D point:
[[3.10058867]
 [0.74321098]
 [0.46490561]]

After reprojection:
[[1069.22527081]
 [ 422.9922658 ]]
[[976.84988144]
 [607.04855241]]
[[1209.96274055]
 [ 435.46606514]]
